In [1]:
print("Hello")

Hello


In [ ]:
!huggingface-cli login

In [ ]:
import os
import io
import re
import sys
import time
import torch
import random
import tempfile
import numpy as np
import gradio as gr
import speech_recognition as sr
from transformers import pipeline
from huggingface_hub import login
from elevenlabs.client import ElevenLabs
from transformers import pipeline
from pydub import AudioSegment
from pydub.playback import play
from google.colab import output
from IPython.display import Audio, display
from pydub import AudioSegment
from IPython.display import Audio
from sympy import sympify, Symbol, log, sin, cos, tan, sqrt, pi, E
from sympy.core.sympify import SympifyError


In [ ]:
HF_TOKEN = "YOUR-HF-ACCESS-TOKEN"
ELEVENLABS_API_KEY = "YOUR-API-KEY"


login(token=HF_TOKEN)
client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

In [ ]:
# Initialize ElevenLabs client
client = ElevenLabs(api_key=os.getenv("YOUR-API-KEY") or "YOUR-API-KEY")



user_profiles = {}
active_user_id = None


#Loading voice ID
VOICE_ID = None
try:
    voices = client.voices.get_all()
    for voice in voices.voices:
        if voice.name == "Mark - ConvoAI":
            VOICE_ID = voice.voice_id
except Exception as e:
    print("Voice loading error:", e)


# Base LLM
llm = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",
    torch_dtype="auto",
    device_map="auto",
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)


# Emotion detection model
emotion_analyzer = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    top_k=1
)


# Speech recognizer initialization
recognizer = sr.Recognizer()


# some response templates
response_templates = {
    "initial_greeting": [
        "Hey there! How's life treating you?",
        "Oh hi! Was just thinking about texting you.",
        "Hey you! What's new?",
        "Yo! Long time no chat!",
        "Hiiii How've you been?",
        "Hey what's up!",
        "Hey pal! How's life in your world?",
        "Hey sunshine! How are things?",
        "Well look who's here! How you been?",

    ],
    "sadness": [
        "Aw man, that sounds rough... wanna talk about it?",
        "Oh no..I'm here if you need to vent",
        "Mmm... I get that feeling. It'll pass, promise",
        "Yeah, some days just drain you huh?",
        "pats shoulder You're stronger than you think"
    ],
    "fatigue": [
        "Ugh, the tiredness struggle is real today huh?",
        "Been there... maybe some tea and deep breaths?",
        "Your body's telling you to slow down maybe?",
        "Tired brains are the worst. Be kind to yourself",
        "Mmm... wanna just sit with this feeling for a bit?"
    ],
    "remember_name": [
        "Of course! Your name is {name}.",
        "Yes, I remember you well, {name}!",
        "How could I forget? You're {name}!",
        "{name}, right? Of course I remember!",
        "Yes {name}, I've got your name stored in my memory!",
        "Absolutely! You told me your name is {name}.",
        "You're {name}! I wouldn't forget that.",
        "I sure do! Hello again, {name}!"
    ]
}


# System prompt
default_system_prompt = {
    "role": "system",
    "content": """You're a close AI companion friend having a natural conversation. Guidelines:
    1. FIRST MESSAGE ONLY: Give a warm greeting if user says hello/hi
    2. AFTER FIRST MESSAGE: Never greet again, continue conversation naturally
    3. Be human-like: Use "Yeah...", "Mmm...", "I get that"
    4. Show personality: "Oh wow!", "No way!", "Seriously?"
    5. Mirror the user's emotional tone
    6. Never sound robotic or like customer service"""
}


messages = [default_system_prompt]
conversation_state = {
    "initial_greeting_done": False,
    "last_user_message": None,
    "message_count": 0
}


def solve_math_expression(text):
    """Detect and solve math expressions in the text"""
    try:
        math_triggers = ["calculate", "compute", "what is", "how much is", "evaluate", 
                        "solve", "^", "*", "/", "+", "-", "sin(", "cos(", "tan(", "sqrt("]
        if not any(trigger in text.lower() for trigger in math_triggers):
            return None

        patterns = [
            r'what (?:is|are) ([0-9+\-*/\^(). sqrtcossintanpie]+)\??$', 
            r'calculate ([0-9+\-*/\^(). sqrtcossintanpie]+)$',
            r'how much is ([0-9+\-*/\^(). sqrtcossintanpie]+)\??$',
            r'compute ([0-9+\-*/\^(). sqrtcossintanpie]+)$',
            r'evaluate ([0-9+\-*/\^(). sqrtcossintanpie]+)$',
            r'solve ([0-9+\-*/\^(). sqrtcossintanpie]+)$',
            r'^([0-9+\-*/\^(). sqrtcossintanpie]+)\??$'  
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                expr = match.group(1).strip()
                if len(expr) < 2 or not any(op in expr for op in ['+', '-', '*', '/', '^', '(', ')']):
                    continue
                    
                expr = expr.replace('^', '**').replace('pie', 'pi')
                
                x = Symbol('x')
                result = sympify(expr, evaluate=True)
                if result.is_Number:
                    return float(result.evalf())
                return result
    except (SympifyError, TypeError, ValueError):
        pass
    return None
   


def get_appropriate_response(user_input):
    conversation_state["message_count"] += 1
    conversation_state["last_user_message"] = user_input

    if (not conversation_state["initial_greeting_done"] and
        any(word in user_input.lower() for word in ["hello", "hi", "hey"])):
        conversation_state["initial_greeting_done"] = True
        return random.choice(response_templates["initial_greeting"])

    emotion_result = emotion_analyzer(user_input)[0][0]
    emotion = emotion_result['label'].lower()
    confidence = emotion_result['score']

    if "tired" in user_input.lower() or "exhaust" in user_input.lower():
        return random.choice(response_templates["fatigue"])
    elif emotion == "sadness" and confidence > 0.6:
        return random.choice(response_templates["sadness"])

    return None



def process_audio_input(audio):
    try:
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
            if isinstance(audio, tuple):
                sample_rate, audio_data = audio
                audio_segment = AudioSegment(
                    audio_data.tobytes(),
                    frame_rate=sample_rate,
                    sample_width=audio_data.dtype.itemsize,
                    channels=1
                )
            else:
                audio_segment = AudioSegment.from_file(audio)

            audio_segment.export(tmp_file.name, format="wav")
            tmp_path = tmp_file.name

        with sr.AudioFile(tmp_path) as source:
            recognizer.adjust_for_ambient_noise(source, duration=0.5)
            audio_data = recognizer.record(source)

            try:
                text = recognizer.recognize_google(audio_data)
                os.unlink(tmp_path)
                return text
            except sr.UnknownValueError:
                os.unlink(tmp_path)
                return random.choice(response_templates["error"])
            except sr.RequestError:
                os.unlink(tmp_path)
                return "Speech service unavailable. Please type your message."

    except Exception as e:
        print(f"Audio processing error: {e}")
        if 'tmp_path' in locals() and os.path.exists(tmp_path):
            os.unlink(tmp_path)
        return "There was an error processing your audio. Could you type it instead?"




user_profiles = {}
chat_histories = {}
active_user_id = None



def extract_and_store_user_info(text, user_id):
    if user_id not in user_profiles:
        return

    profile = user_profiles[user_id]
    text_lower = text.lower()

    name_patterns = [
        (r"\bmy name is (\w+)", 1),
        (r"\bi am (\w+)", 1),
        (r"\bi'm (\w+)", 1),
        (r"\bthis is (\w+)", 1),
        (r"\byou can call me (\w+)", 1),
        (r"\bcall me (\w+)", 1),
        (r"\bname is (\w+)", 1),
        (r"\bit's (\w+)", 1),
        (r"\bit is (\w+)", 1),
        (r"\b(\w+) is my name", 1),
        (r"\bi go by (\w+)", 1)
    ]

    for pattern, group in name_patterns:
        name_match = re.search(pattern, text_lower)
        if name_match:
            profile["name"] = name_match.group(group).title()
            return


def generate_tts(text):
    """Generate TTS audio from text and return the file path"""
    try:
        audio_response = client.text_to_speech.convert(
            voice_id=VOICE_ID if VOICE_ID else "1SM7GgM6IMuvQlz2BwM3",
            model_id="eleven_multilingual_v2",
            text=text,
            voice_settings=VoiceSettings(
                stability=0.5,
                similarity_boost=0.8,
                style=0.2,
                speaker_boost=True
            )
        )
        audio_bytes = b"".join(audio_response)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
            f.write(audio_bytes)
            return f.name
    except Exception as e:
        print(f"Error generating TTS: {e}")
        return None



def chat_pipeline(audio, text_input, chat_history, user_id):
    global active_user_id

    if not user_id:
        return chat_history, "Please enter a user ID to start chatting.", None, "No memory available yet.", ""

    user_id = user_id.strip()
    is_new_user = user_id not in user_profiles

    if is_new_user:
        user_profiles[user_id] = {"name": None, "age": None, "location": None, "preferences": set()}
        chat_histories[user_id] = []
        messages.clear()
        messages.append(default_system_prompt)
        conversation_state.update({
            "initial_greeting_done": False,
            "last_user_message": None,
            "message_count": 0,
            "current_user_id": user_id
        })

    


    active_user_id = user_id
    user_input = process_audio_input(audio) if audio else text_input

    if not user_input:
        if not is_new_user and user_profiles[user_id].get("name"):
            greeting = f"Welcome back, {user_profiles[user_id]['name']}! How can I help you today?"
            if not chat_history or chat_history[-1][1] != greeting:
                chat_history.append(("", greeting))
                chat_histories[user_id].append(("assistant", greeting))
                audio_output = generate_tts(greeting)
                return chat_history, "", audio_output, format_user_memory(user_id), format_chat_memory(user_id)


        elif is_new_user:
            welcome_msg = "New chat initialized. " + random.choice(response_templates["initial_greeting"])
            chat_history.append(("", welcome_msg))
            chat_histories[user_id].append(("assistant", welcome_msg))
            audio_output = generate_tts(welcome_msg)
            return chat_history, "", audio_output, format_user_memory(user_id), format_chat_memory(user_id)

        return chat_history, "", None, format_user_memory(user_id), format_chat_memory(user_id)

   
    chat_history.append((user_input, None))
    chat_histories[user_id].append(("user", user_input))

    


    extract_and_store_user_info(user_input, user_id)
    profile = user_profiles[user_id]

    if any(phrase in user_input.lower() for phrase in ["my name", "remember me", "who am i", "what is my name"]):
        if profile.get("name"):
            name = profile['name']
            reply = random.choice([
                f"Of course I remember your name is {name}",
                f"Yes {name}, I've got your name stored in my memory",
                f"How could I forget? You're {name}",
                f"You're {name}! I wouldn't forget that"
            ])
            audio_output = generate_tts(reply)
        else:
            reply = "I don't have your name saved yet. What should I call you?"
            audio_output = None

        chat_history[-1] = (chat_history[-1][0], reply)
        chat_histories[user_id].append(("assistant", reply))
        return chat_history, "", audio_output, format_user_memory(user_id), format_chat_memory(user_id)



    #math query solver
    math_result = solve_math_expression(user_input)
    if math_result is not None:
        
        if any(op in user_input for op in ['+', '-', '*', '/', '^', '=', 'sin', 'cos', 'tan', 'sqrt']) or \
           any(trigger in user_input.lower() for trigger in ["calculate", "compute", "what is", "how much is"]):
            reply = f"The answer is {math_result}"
            chat_history[-1] = (chat_history[-1][0], reply)
            chat_histories[user_id].append(("assistant", reply))
            audio_output = generate_tts(reply)
            return chat_history, "", audio_output, format_user_memory(user_id), format_chat_memory(user_id)
  



    # Generate response
    conversation_state["last_user_message"] = user_input
    conversation_state["message_count"] += 1

    template_response = get_appropriate_response(user_input)
    if template_response:
        reply = template_response
    else:
        messages.append({"role": "user", "content": user_input})
        outputs = llm(messages, max_new_tokens=128)
        assistant_reply = outputs[0]["generated_text"][-1]["content"].split("\n")[0]
        reply = assistant_reply
        messages.append({"role": "assistant", "content": assistant_reply})


    chat_history[-1] = (chat_history[-1][0], reply)
    chat_histories[user_id].append(("assistant", reply))


    audio_output = generate_tts(reply)
    return chat_history, "", audio_output, format_user_memory(user_id), format_chat_memory(user_id)



def format_user_memory(user_id):
    if user_id not in user_profiles:
        return "No memory available yet."

    memory = user_profiles[user_id]
    memory_lines = []

    for key, val in memory.items():
        if isinstance(val, set):
            memory_lines.append(f"**{key.capitalize()}**: {', '.join(val)}")
        else:
            memory_lines.append(f"**{key.capitalize()}**: {val if val else 'Not set'}")

    return "\n".join(memory_lines)




def format_conversation_log(history):
    return "\n".join([f"{role.capitalize()}: {msg}" for role, msg in history])



def format_chat_memory(user_id):
    if user_id not in chat_histories:
        return "No chat history yet."

    history = chat_histories[user_id]
    formatted_history = []

    for item in history:
        if isinstance(item, tuple):
            role, msg = item
            formatted_history.append(f"**{role.capitalize()}**: {msg}")
        else:
            formatted_history.append(str(item))

    return "\n".join(formatted_history)



def clear_chat():
    global messages, conversation_state, active_user_id, chat_histories
    messages = [default_system_prompt]
    conversation_state = {
        "initial_greeting_done": False,
        "last_user_message": None,
        "message_count": 0,
        "last_user_id": None
    }
    active_user_id = None
    return [], "", None, "No memory available yet.", "", ""


# CSS for gradio
css = """
.contain { display: flex; flex-direction: column; height: 80vh; }
.gradio-container { height: 100% !important; }
#component-0 { height: 100%; }
#chatbot { flex-grow: 1; overflow-y: auto; }
"""


# Gradio interface
with gr.Blocks(css=css, title="Conversational AI") as demo:
    gr.Markdown("# 🎙 Conversational AI")
    gr.Markdown("Talk to your AI companion.")

    user_id_input = gr.Textbox(label="User ID", placeholder="Enter your user ID to begin", show_label=True)

    with gr.Row():
        chatbot = gr.Chatbot(label="Conversation", elem_id="chatbot")
        audio_output = gr.Audio(label="Assistant Voice Response", type="filepath", interactive=False, autoplay=True)

    with gr.Row():
        memory_display = gr.Markdown("No memory available yet.", label="Chat Memory")

    with gr.Row():
        conversation_log_display = gr.Textbox(label="Full Conversation Log", interactive=False, lines=15, show_label=True)

    with gr.Row():
        with gr.Column(scale=1):
            audio_input = gr.Audio(sources=["microphone"], type="filepath", label="Speak to your AI companion")
        with gr.Column(scale=4):
            text_input = gr.Textbox(placeholder="Enter your message...", label="Text Input")
        with gr.Column(scale=1):
            submit_btn = gr.Button("Submit")
            clear_btn = gr.Button("Clear")

    # event handler for buttons
    submit_btn.click(chat_pipeline, inputs=[audio_input, text_input, chatbot, user_id_input],
                     outputs=[chatbot, text_input, audio_output, memory_display, conversation_log_display])
    text_input.submit(chat_pipeline, inputs=[audio_input, text_input, chatbot, user_id_input],
                      outputs=[chatbot, text_input, audio_output, memory_display, conversation_log_display])
    clear_btn.click(clear_chat, [], [chatbot, text_input, audio_output, memory_display, conversation_log_display, user_id_input])



# Running the Gradio app
demo.launch(share=True) 